# Imports

In [12]:
from dataProcessing import getTextsInDateRange, getPairs, getFullWordset, buildTokenizer
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense

import random
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /Users/ebolton/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Conda env checking
From article here:
https://towardsdatascience.com/get-your-conda-environment-to-show-in-jupyter-notebooks-the-easy-way-17010b76e874

If conda env is being used correctly, the environment variable below should be `base`.

In [2]:
import os
print(os.environ['CONDA_DEFAULT_ENV'])

base


# Constants

In [3]:
CHANNEL_MAPPINGS = {
#     "Daily Wire": "UCroKPvbmaQKGK5tjtQsvaDw",
    "Ben Shapiro": "UCnQC_G5Xsjhp9fEJKuIcrSw",
#     "Daily Wire Plus": "UCaeO5vkdj5xOQHp4UmIN6dw",
#     "Matt Walsh": "UCO01ytfzgXYy4glnPJm4PPQ",
#     "Michael Knowls": "UCr4kgAUTFkGIwlWSodg43QA",
#     "Candace Owens": "UCkY4fdKOFk3Kiq7g5LLKYLw"
}

# Parameters

In [4]:
# channels = ['Ben Shapiro']
channels = list(CHANNEL_MAPPINGS.keys())
start = 2021
stop = 2023

# Get Texts

In [5]:
nltk.download('omw-1.4')
channelIds = [CHANNEL_MAPPINGS[c] for c in channels]
texts = getTextsInDateRange(channelIds, start, stop, cleaned=True)
channelstext = "\n  - ".join(channels)
print([t[:30] for t in texts[:10]])
print()
print(f'Found {len(texts)} videos for channels: \n  - {channelstext}')

[nltk_data] Downloading package omw-1.4 to /Users/ebolton/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


[['sad', 'left', 'california', 'tell', 'thought', 'one', 'day', 'decision', 'one', 'best', 'decision', 'ever', 'made', 'family', 'know', 'orthodox', 'jewish', 'faith', 'mean', 'move', 'place', 'significant', 'jewish', 'resource', 'inundation', 'homelessness', 'area', 'like', 'kid', 'could'], ['bizarre', 'new', 'world', 'created', 'counter-reality', 'new', 'world', "we've", 'created', 'assigning', 'sex', 'baby', 'dc', 'forcing', 'label', 'mother', "that's", 'case', 'throne', 'mother', 'whole', 'birth', 'certificate', 'thing', 'really', 'really', 'causing', 'lot', 'oh', 'i'], ['end', 'entire', 'group', 'people', 'single', 'woman', 'falling', 'behind', 'married', 'woman', 'engaged', 'social', 'structure', 'important', 'mediating', 'institution', 'human', 'existence', 'marriage', 'ripped', 'away', "they've", 'told', 'longer', 'essential', 'important', 'fact', 'many', 'way', 'institution'], ['idea', 'left', 'everybody', 'right', 'keep', 'saying', "there's", 'election', 'fraud', "that's", 'r

# Build Pairs

In [6]:
pairs = getPairs(texts, window=2)
print(pairs[:10])
print(random.sample(pairs, 100))
print(f'Built {len(pairs)} pairs')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 583/583 [00:00<00:00, 834.73it/s]

[('sad', 'left'), ('sad', 'california'), ('left', 'california'), ('left', 'sad'), ('left', 'tell'), ('california', 'tell'), ('california', 'left'), ('california', 'thought'), ('california', 'sad'), ('tell', 'thought')]
[('favorite', 'democrat'), ('photo', 'image'), ('listener', 'allform.com'), ('clear', 'thing'), ('issuing', 'letter'), ('marbled', 'fat'), ('build', 'scratch'), ("what's", 'joe'), ('merit', 'letter'), ('coalition', 'right'), ('idea', 'tax'), ('police', 'promoting'), ('disclosure', 'said'), ('know', 'company'), ('deserves', 'get'), ('left', 'lot'), ('troop', 'buildup'), ('hair', 'face'), ('possible', 'daily'), ('sort', 'presence'), ('long', 'remains'), ('really', 'talk'), ('perch', 'matcha'), ('really', 'think'), ('forcing', 'everybody'), ('car', 'hit'), ('shapiro', 'welcome'), ('alliance', 'party'), ('statement', 'would'), ('like', 'first'), ('brown', 'contact'), ('worth', 'haqqani'), ('pure', 'today'), ('joe', 'federal'), ('left', 'signaling'), ('taken', 'vichy'), ('hun

# Get wordset and build tokenizer

In [7]:
wordset = getFullWordset(texts)
word2idx, idx2word = buildTokenizer(wordset)
print(list(wordset)[:20])
print(random.sample(list(wordset), 20))
print(f'Wordset is size: {len(wordset)}')

['', 'urumiki', 'gearing', '2022', '9.', 'intolerance', 'engaging', 'mastercard', 'muffler', 'empirically', 'obesity', 'alright', 'spilling', 'averted', 'emil', 'soil', 'fdic', 'lubich', "airsat's", '467']
['eyed', 'clayne', 'characterization', 'd-day', 'pickle', 'minivan', 'matt', 'anti-military', 'vector', 'penguin', 'saved', 'sure', 'cosplaying', '1987', 'eh', 'crunch', 'clever', 'subordinate', 'try', 'steamrolled']
Wordset is size: 30493


# Convert Pairs to ints

In [9]:
def pairToInts(pair, mapping):
    a, b = pair
    return (mapping[a], mapping[b])

pairset = set(pairs)
intpairs = []
# intpairs = [pairToInts(p, word2idx) for p in pairs]
for p in pairset:
    intpairs.append(pairToInts(p, word2idx))
print(intpairs[:10])
print(weights[:10])
print(f'Total pairs: {len(intpairs)}')

[(24210, 26980), (25116, 26812), (4628, 4936), (19035, 13852), (29881, 582), (22682, 22180), (4135, 6902), (14018, 13243), (7828, 20018), (4018, 14790)]
[]
Total pairs: 1775754


# Count pairs for creating weights

In [10]:
pairWeights = {}
for pair in intpairs:
    if pair not in pairWeights:
        pairWeights[pair] = 0
    pairWeights[pair] += 1

# Define generating function for data

In [13]:
x = np.array(tf.one_hot(5, 10), dtype=np.bool_)
print(x)
print(x.shape)
print(x.dtype)

Metal device set to: Apple M1 Max
[False False False False False  True False False False False]
(10,)
bool


2022-12-12 18:14:06.270453: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-12 18:14:06.270800: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [55]:
def generateData():
    count = 0
    for p in intpairs:
        count += 1
        if count > 10:
            break
        w = pairWeights[p]
        a, b = p
        va = tf.one_hot(a, len(wordset), dtype=tf.uint8)
        vb = tf.one_hot(b, len(wordset), dtype=tf.uint8)
        # yield (va, vb, w)
        yield (va, vb)



In [56]:
x = list(generateData())
print(x)
print(len(x))

[(<tf.Tensor: shape=(30493,), dtype=uint8, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)>, <tf.Tensor: shape=(30493,), dtype=uint8, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)>), (<tf.Tensor: shape=(30493,), dtype=uint8, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)>, <tf.Tensor: shape=(30493,), dtype=uint8, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)>), (<tf.Tensor: shape=(30493,), dtype=uint8, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)>, <tf.Tensor: shape=(30493,), dtype=uint8, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)>), (<tf.Tensor: shape=(30493,), dtype=uint8, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)>, <tf.Tensor: shape=(30493,), dtype=uint8, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)>), (<tf.Tensor: shape=(30493,), dtype=uint8, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)>, <tf.Tensor: shape=(30493,), dtype=uint8, numpy=array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)>), (<tf.Tensor: shape=(30493,), dtype=uint8, numpy=array([0, 0

# Build and compile NNet

In [57]:
EMBED_SIZE = 64
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [58]:
inp = Input(shape=(len(wordset),))
x = Dense(units=EMBED_SIZE, activation='linear')(inp)
x = Dense(units=len(wordset), activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [59]:
print(model.summary())

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 30493)]           0         
                                                                 
 dense_4 (Dense)             (None, 64)                1951616   
                                                                 
 dense_5 (Dense)             (None, 30493)             1982045   
                                                                 
Total params: 3,933,661
Trainable params: 3,933,661
Non-trainable params: 0
_________________________________________________________________
None


# Train NNet

In [60]:
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [61]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [63]:
# hist = model.fit(
#     x=X,
#     y=Y,
#     batch_size=32,
#     epochs=150,
#     sample_weight=trainingWeights
# )
gen = tf.data.Dataset.from_generator(generateData, (tf.uint8, tf.uint8))
print(gen)
hist = model.fit(
    gen,
    epochs=2,
)

<FlatMapDataset element_spec=(TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None), TensorSpec(shape=<unknown>, dtype=tf.uint8, name=None))>
Epoch 1/2


2022-12-12 18:50:20.132618: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-12-12 18:50:20.382341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


InvalidArgumentError: Graph execution error:

Detected at node 'model_2/dense_4/MatMul' defined at (most recent call last):
    File "/Users/ebolton/miniconda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/ebolton/miniconda/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/ebolton/miniconda/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/ebolton/miniconda/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/ebolton/miniconda/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/ebolton/miniconda/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/Users/ebolton/miniconda/lib/python3.9/async

In [71]:
# Define a generator function that yields data
def my_generator():
  for i in range(10):
    yield (i, i*2)

# Create a Dataset object from the generator
dataset = tf.data.Dataset.from_generator(my_generator, (tf.int32, tf.int32))

# Use the Dataset in a TensorFlow model
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(10, input_shape=(1,)),
  tf.keras.layers.Dense(1)
])
model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mse')
print(list(my_generator()))
print(model.summary())
for layer in model.layers:
    print(layer.input_shape, layer.output_shape)
model.fit(dataset, epochs=3)

[(0, 0), (1, 2), (2, 4), (3, 6), (4, 8), (5, 10), (6, 12), (7, 14), (8, 16), (9, 18)]
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_20 (Dense)            (None, 10)                20        
                                                                 
 dense_21 (Dense)            (None, 1)                 11        
                                                                 
Total params: 31
Trainable params: 31
Non-trainable params: 0
_________________________________________________________________
None
(None, 1) (None, 10)
(None, 10) (None, 1)
Epoch 1/3


2022-12-12 18:57:36.311253: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2022-12-12 18:57:36.362218: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at strided_slice_op.cc:111 : INVALID_ARGUMENT: slice index 0 of dimension 0 out of bounds.


InvalidArgumentError: Graph execution error:

Detected at node 'strided_slice' defined at (most recent call last):
    File "/Users/ebolton/miniconda/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/Users/ebolton/miniconda/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/Users/ebolton/miniconda/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/Users/ebolton/miniconda/lib/python3.9/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/Users/ebolton/miniconda/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/Users/ebolton/miniconda/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/Users/ebolton/miniconda/lib/python3.9/asyncio/base_e

In [48]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')
# tf.test.is_gpu_available()

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Performance Stats

In [ ]:
import matplotlib.pyplot as plt
plt.plot(hist.history['loss'])
plt.show()

# Build word -> vector mapping

In [ ]:
weights = model.get_weights()[0]
print(weights)
print(weights.shape)
word2vec = {}
for word in wordset:
    vec = weights[word2idx[word]]
    word2vec[word] = vec

# Check pair counts

In [ ]:
print(len(pairs))
print(len(set(pairs)))
s = set(pairs)
counts = {}
for p in pairs:
    if p not in counts:
        counts[p] = 1
    else:
        counts[p] = counts[p]+1
sortedpairs = sorted(list(s), key=lambda p: counts[p], reverse=True)
for p in sortedpairs:
    print(counts[p], p)

# Get nearest neighbors

In [ ]:
targetWord = 'Biden'
v = word2vec[targetWord]
print(v.shape)

dists = {}
for word, vec in word2vec.items():
    d = np.linalg.norm(v - vec)
    dists[word] = d

allwords = list(wordset)
allwords = sorted(allwords, key=lambda w: dists[w])
N = 10
for i in range(N):
    d = dists[allwords[i]]
    print(f"{i:<2} {str(round(d, 3)):<6} {allwords[i]}")

# Export wordset

In [ ]:
wordtext = '\n'.join(wordset)
with open('wordset.txt', 'w') as file:
    file.write(wordtext)
    print('WROTE FILE')